In [1]:
import support_functions
import polars as pl
import torch
import data_download
from torch.utils.data import DataLoader, TensorDataset

did not download


In [2]:
def l2_normalize(tensor, dim=0, epsilon=1e-12):
            """
            Applies L2 normalization to a tensor along the specified dimension.

            Args:
            tensor (torch.Tensor): The input tensor to normalize.
            dim (int): The dimension along which to normalize. Default is 0.
            epsilon (float): A small value to avoid division by zero. Default is 1e-12.

            Returns:
            torch.Tensor: The L2-normalized tensor.
            """
            # Calculate the L2 norm along the specified dimension, keeping dimensions
            l2_norm = torch.norm(tensor, p=2, dim=dim, keepdim=True)
            
            # Prevent division by zero by adding a small value epsilon to the norm
            l2_norm = torch.clamp(l2_norm, min=epsilon)
            
            # Divide the tensor by its L2 norm
            normalized_tensor = tensor / l2_norm
            
            return normalized_tensor

In [3]:
dbname='ai_dataset'
dbuser='postgres'
dbpassword='parola'
dbport=5432
dbhost='127.0.0.1'
table_name='tabela'


tickers = ["AAPL", "atun.INDX", 'SPY', 'XAUUSD.OANDA', 'BCO.ICMTRADER','NVDA']
for i in range(len(tickers)):
    tickers[i]=tickers[i].replace('.','_')
net_functions=support_functions.Protected_execution

In [4]:
query=net_functions.create_query(tickers[0],table_name)
cursor,conn=net_functions.create_cursor(dbname,dbuser,dbpassword,dbport,dbhost)
data=pl.read_database(query=query,connection=conn)
data=data.drop_nulls()


In [5]:
data

aapl_open,aapl_high,aapl_low,aapl_close,aapl_adj_close,aapl_volume,t
f64,f64,f64,f64,f64,f64,f64
177.83,182.88,177.71,179.0766,179.0766,1.04487904e8,1.6412e9
182.63,182.94,179.12,176.8038,176.8038,9.93104e7,1.6413e9
179.61,180.17,174.64,172.1009,172.1009,9.45376e7,1.6413e9
172.7,175.3,171.64,169.2279,169.2279,9.6904e7,1.6414e9
172.89,174.14,171.03,169.3952,169.3952,8.6709104e7,1.6415e9
…,…,…,…,…,…,…
228.06,230.72,228.06,229.87,229.87,3.81683e7,1.7322e9
231.46,233.25,229.74,232.87,232.87,9.01528e7,1.7325e9
233.58,235.57,233.33,235.06,235.06,4.59862e7,1.7326e9


In [6]:
time=data['t']

In [7]:
from datetime import datetime

In [8]:
pl.from_epoch(time, time_unit='s')

t
datetime[μs]
2022-01-03 00:00:00
2022-01-04 00:00:00
2022-01-05 00:00:00
2022-01-06 00:00:00
2022-01-07 00:00:00
…
2024-11-22 00:00:00
2024-11-25 00:00:00
2024-11-26 00:00:00


In [9]:
seconds=pl.from_epoch(time,time_unit="s")

In [10]:
date=pl.from_epoch(seconds,time_unit='d')

In [11]:
days=date
days

t
date
2022-01-03
2022-01-04
2022-01-05
2022-01-06
2022-01-07
…
2024-11-22
2024-11-25
2024-11-26


In [12]:
type(days)

polars.series.series.Series

In [13]:
days=days.rename('date')

In [14]:
frame=days.to_frame(days)

In [15]:
data=pl.concat([data,frame],how='horizontal')


In [16]:
frame 

date
date
2022-01-03
2022-01-04
2022-01-05
2022-01-06
2022-01-07
…
2024-11-22
2024-11-25
2024-11-26


In [17]:
#context=['VIX.INDX','SPY']
context=['vix_indx','spy']
for index in context:
    query=net_functions.create_query(index,table_name,add=True)
    cont=pl.read_database(query=query,connection=conn)
    cont=cont.drop_nulls()
    var=cont['t']    
    var=pl.from_epoch(var)
    var=pl.from_epoch(var,time_unit='d')
    var=var.to_frame(var)
    cont=cont.drop('t')
    cont=pl.concat([cont,var],how='horizontal')
    cont=cont.rename({"t":"date"})
    data=data.join(cont,on='date')



In [18]:
data=data.drop('date')
data=data.drop_nulls()
data

aapl_open,aapl_high,aapl_low,aapl_close,aapl_adj_close,aapl_volume,t,vix_indx_close,spy_close
f64,f64,f64,f64,f64,f64,f64,f64,f64
177.83,182.88,177.71,179.0766,179.0766,1.04487904e8,1.6412e9,16.6,470.0829
182.63,182.94,179.12,176.8038,176.8038,9.93104e7,1.6413e9,16.91,469.9254
179.61,180.17,174.64,172.1009,172.1009,9.45376e7,1.6413e9,19.73,460.9018
172.7,175.3,171.64,169.2279,169.2279,9.6904e7,1.6414e9,19.61,460.4689
172.89,174.14,171.03,169.3952,169.3952,8.6709104e7,1.6415e9,18.76,458.6484
…,…,…,…,…,…,…,…,…
228.06,230.72,228.06,229.87,229.87,3.81683e7,1.7322e9,15.24,595.51
231.46,233.25,229.74,232.87,232.87,9.01528e7,1.7325e9,14.6,597.53
233.58,235.57,233.33,235.06,235.06,4.59862e7,1.7326e9,14.1,600.65


In [19]:
train_data=data['aapl_close']
labels=net_functions.change(train_data)
train_data=torch.tensor(train_data)
labels=torch.Tensor(labels)
train_data=train_data[:-1]
batch_size=32
shortened_tensor=net_functions.tensor_shortner(train_data, batch_size)
image_tensor=net_functions.image_builder(shortened_tensor,batch_size=batch_size)
image_tensor=image_tensor[:-1]
labels=torch.Tensor(labels)
labels=net_functions.tensor_shortner(labels,batch_size)
labels=[x+5 for x in labels]
labels=torch.tensor(labels,dtype=torch.float32)
image_tensor=torch.tensor(image_tensor)
image_tensor=net_functions.make_tensor(data,image_tensor,tickers[0])
temp_labels=net_functions.image_builder(labels[:-1],32)
temp_labels=torch.Tensor(temp_labels)
image_tensor=torch.cat((image_tensor,temp_labels),dim=1)
labels=labels[batch_size+2:]
labels=labels.float()

for i in range(len(image_tensor[0].size())):
    image_tensor[i]=l2_normalize(image_tensor[i])
train_dataset = TensorDataset(image_tensor, labels)
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=False)
